In [1]:
import hashlib
from bitarray import bitarray
from Cryptodome.Util import number
from merklelib import MerkleTree

# a sample hash function
# you can also omit it and the default hash function will be used
def split_in_128(value):
    unpadded = list(str(bin(value)[2:]))
    padded = "".join(map(str, [0] * (256 - len(unpadded)) + unpadded))
    value1, value2 = padded[:128], padded[128:]
    return int(value1, 2), int(value2, 2)

def printhash(value):
    print(f'''
    {'-' * 20}
    number: {value}
    hexdigest: {hex(value)}
    number size: {number.size(value)}
    128 pairs: {split_in_128(value)}
    ''')
    
def unpack256(value):
    return (value).to_bytes(32, byteorder='big')

def unpack512(value):
    return (value).to_bytes(64, byteorder='big')

def combine_128s(a, b):
    return (a << 128) + b
    
def leafhash(lhs, rhs):
    preimage = unpack256(lhs) +  unpack256(rhs)
    hex_hash = hashlib.sha256(preimage).hexdigest()
    hex_num = int(hex_hash,16)
    printhash(hex_num)
    return hex_num

def treehash(lhs, rhs):
    preimage = unpack512(lhs + rhs)
    hex_hash = hashlib.sha256(preimage).hexdigest()
    hex_num = int(hex_hash,16)
    printhash(hex_num)
    return hex_num

def format_barray(barray):
    barray_str = ''.join(barray)
    hashnum = int(barray_str, 2)
    printhash(hashnum)
    return hashnum


In [2]:
c = 20075373234943686845167158285967784892467090849631486320124245130906619831459
p = 21888242871839275222246405745257275088548364400416034343698204186575808495619
print(number.size(p))
x = 10340591486849500411597554648864878801016182183377116661129044131080489564292
y = 14567100831092071334990535999461575549871942024819092424591682457990310750003
number.size(x + y)
p > x + y
x + y

# test hash function
padded_five = ["1","1","0","0","0","1","1","0","0","1","0","0","1","0","0","0","0","0","0","1","1","1","1","0","0","0","1","0","0","0","1","0","1","1","0","0","0","1","0","1","1","1","1","1","1","1","1","1","0","1","0","0","0","0","0","1","0","1","1","0","0","1","0","0","1","0","1","0","1","1","1","1","0","1","1","0","1","0","0","0","0","0","0","0","1","0","1","1","1","0","0","0","1","1","0","0","1","1","1","1","1","0","1","0","1","0","1","0","0","1","0","1","1","1","1","0","1","0","0","0","1","1","1","0","1","1","0","1","0","0","1","1","0","0","0","1","0","0","1","0","0","0","0","0","1","1","1","0","1","1","1","0","1","1","1","1","1","1","1","1","1","0","0","0","1","0","0","1","1","1","0","0","0","1","0","0","1","1","1","1","0","0","1","1","0","0","0","0","0","1","1","1","1","1","0","0","0","1","0","0","1","0","1","0","0","1","1","0","1","1","1","1","1","0","1","0","1","0","1","0","1","0","1","0","1","1","1","0","0","0","0","0","0","1","0","1","1","0","0","1","1","1","0","0","1","1","1","0","0","0","0","1","0","0","0","0"]
padded_zero = ["1","1","1","1","0","1","0","1","1","0","1","0","0","1","0","1","1","1","1","1","1","1","0","1","0","1","0","0","0","0","1","0","1","1","0","1","0","0","0","1","0","1","1","0","1","0","1","0","0","0","1","0","0","0","0","0","0","0","1","1","0","0","0","0","0","0","1","0","0","1","1","1","1","0","0","1","1","0","0","0","1","1","1","0","1","1","1","1","0","1","1","0","1","1","1","0","1","1","0","1","0","0","1","1","0","0","0","0","1","0","0","1","1","0","0","1","0","1","1","1","1","0","0","1","1","0","1","1","0","1","0","0","0","0","1","1","0","0","0","0","0","0","0","0","0","0","1","1","1","1","0","1","0","0","1","0","0","0","1","1","0","0","1","0","0","0","0","0","1","1","0","1","1","0","0","1","1","1","1","1","0","0","0","0","1","1","1","0","1","0","0","0","1","1","1","0","1","0","1","0","1","0","0","1","1","0","0","0","0","0","1","1","0","0","0","1","1","0","1","0","1","0","0","1","0","0","1","0","0","1","1","1","0","1","0","1","1","0","0","1","1","1","1","1","1","0","1","1","0","1","0","0","1","0","1","1"]

assert format_barray(padded_five) == leafhash(0, 5)
assert format_barray(padded_zero) == leafhash(0, 0)

254

    --------------------
    number: 89685364998030906426902553293848047120578154677247506650664740170569575157264
    hexdigest: 0xc6481e22c5ff4164af680b8cfaa5e8ed3120eeff89c4f307c4a6faaae059ce10
    number size: 256
    128 pairs: (263561599766550617289250058199814760685, 65303172752238645975888084098459749904)
    

    --------------------
    number: 89685364998030906426902553293848047120578154677247506650664740170569575157264
    hexdigest: 0xc6481e22c5ff4164af680b8cfaa5e8ed3120eeff89c4f307c4a6faaae059ce10
    number size: 256
    128 pairs: (263561599766550617289250058199814760685, 65303172752238645975888084098459749904)
    

    --------------------
    number: 111109925611824843164212799849330761292948257037696933205019304127221294824267
    hexdigest: 0xf5a5fd42d16a20302798ef6ed309979b43003d2320d9f0e8ea9831a92759fb4b
    number size: 256
    128 pairs: (326522724692461750427768532537390503835, 89059515727727869117346995944635890507)
    

    --------------------
    nu

In [3]:
pk = (str(10340591486849500411597554648864878801016182183377116661129044131080489564292)
      , str(14567100831092071334990535999461575549871942024819092424591682457990310750003))
sk = str(127701493367712430457058180578738495689)

In [4]:
leafDigest = leafhash(x, y)
l_1, l_2 = split_in_128(leafDigest)
assert leafDigest == combine_128s(l_1, l_2)

pathDigest0 = leafhash(0, 0)
pd0_1, pd0_2 = split_in_128(pathDigest0)
assert pathDigest0 == combine_128s(pd0_1, pd0_2)

pathDigest1 = leafhash(0, 5)
pd1_1, pd1_2 = split_in_128(pathDigest1)
assert pathDigest1 == combine_128s(pd1_1, pd1_2)

pathDigest2 = leafhash(0, 7)
pd2_1, pd2_2 = split_in_128(pathDigest2)
assert pathDigest2 == combine_128s(pd2_1, pd2_2)


    --------------------
    number: 42109196106316939627359049747152562481204628433571261653570481959064075030769
    hexdigest: 0x5d18f5dbfb0469a7136a3f3f95f46b3b1dbf434e1f9d95574ed8d40f8a40d0f1
    number size: 255
    128 pairs: (123747805351608569425204621146108226363, 39540705688767375067091866303177216241)
    

    --------------------
    number: 111109925611824843164212799849330761292948257037696933205019304127221294824267
    hexdigest: 0xf5a5fd42d16a20302798ef6ed309979b43003d2320d9f0e8ea9831a92759fb4b
    number size: 256
    128 pairs: (326522724692461750427768532537390503835, 89059515727727869117346995944635890507)
    

    --------------------
    number: 89685364998030906426902553293848047120578154677247506650664740170569575157264
    hexdigest: 0xc6481e22c5ff4164af680b8cfaa5e8ed3120eeff89c4f307c4a6faaae059ce10
    number size: 256
    128 pairs: (263561599766550617289250058199814760685, 65303172752238645975888084098459749904)
    

    --------------------
    number

In [5]:
level0 = leafhash(leafDigest, pathDigest0)
currentDigest0 = split_in_128(level0)
assert level0 == combine_128s(currentDigest0[0], currentDigest0[1])

level1 = leafhash(level0, pathDigest1)
currentDigest1 = split_in_128(level1)
assert level1 == combine_128s(currentDigest1[0], currentDigest1[1])

level2 = leafhash(level1, pathDigest2)
currentDigest2 = split_in_128(level2)
assert level2 == combine_128s(currentDigest2[0], currentDigest2[1])


    --------------------
    number: 19053794382149428646011151054074259967247695083745939853369937676780738122262
    hexdigest: 0x2a2010c1297a34c21e688d31d509329a53052b4701f21f0fa492f338eb4b1216
    number size: 254
    128 pairs: (55994069144865228322401559686429618842, 110352762903855378863843860946600268310)
    

    --------------------
    number: 98185497375201814532797324616020809174812337513488548543275754978356515442187
    hexdigest: 0xd91305abb95987f653f9c969f94a4c30537ac4e636d48d09b8ee239bdb020e0b
    number size: 256
    128 pairs: (288541243743065677620941918019988900912, 110963377458618109267028790611068194315)
    

    --------------------
    number: 104223561982636764801498285708342372631755568568717773728474766327916101504808
    hexdigest: 0xe66c72138685d9381d1d9dad0bddad3dc2531c98fd1fb2468650323de7da0f28
    number size: 256
    128 pairs: (306285520832915120451919103830098029885, 258301771850019700518850468320414142248)
    


In [9]:
pair_to_str = lambda x: " ".join([f'{i[0]} {i[1]}' for i in x])
root_digest = pair_to_str([currentDigest2])
digests = [(l_1, l_2), (pd0_1, pd0_2), (pd1_1, pd1_2), (pd2_1, pd2_2)]
path_digests = pair_to_str(digests)
directions = " ".join(['0'] * (len(digests) - 1))
#print(f'{root_digest}')
print(f'zokrates compute-witness -a {pk[0]} {pk[1]} {sk} {directions} {path_digests} {root_digest}')

zokrates compute-witness -a 10340591486849500411597554648864878801016182183377116661129044131080489564292 14567100831092071334990535999461575549871942024819092424591682457990310750003 127701493367712430457058180578738495689 0 0 0 123747805351608569425204621146108226363 39540705688767375067091866303177216241 326522724692461750427768532537390503835 89059515727727869117346995944635890507 263561599766550617289250058199814760685 65303172752238645975888084098459749904 62133134181886812829768166950054220896 160635334427203623512968684759912538624 306285520832915120451919103830098029885 258301771850019700518850468320414142248


In [7]:
# a list of all ASCII letters
data = [
    leafDigest,
]

# build a Merkle tree for that list
# tree = MerkleTree(data, hashfunc)

# # generate an audit proof the letter A
# proof = tree.get_proof('A')

# # now verify that A is in the tree
# # you can also pass in the hash value of 'A'
# # it will hash automatically if the user forgot to hash it
# if tree.verify_leaf_inclusion('A', proof):
#     print('A is in the tree')
# else:
#     exit('A is not in the tree')

In [8]:
tree_root == leafhash(leafDigest, pathDigest0)

NameError: name 'tree_root' is not defined

In [ ]:
print(f'{e} {f} {a} {b} 0 {c} {d}')

In [ ]:
str(split_in_128(tree_root))